In [3]:
from pyspark import SparkContext, SparkConf

conf = (SparkConf()
         .setMaster("local")
         .setAppName("News Processing")
         .set("spark.executor.memory", "9g"))
            
sc = SparkContext(conf = conf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=News Processing, master=local) created by __init__ at <ipython-input-1-da79fb8e4351>:8 

In [4]:
import re
import nltk
import string
import pandas as pd
import news_config as config
from pymystem3 import Mystem
from NewsMongoMiner import NewsMongoHelper

import sys
reload(sys)
sys.setdefaultencoding('utf8')

%load_ext autoreload
%autoreload 2

In [6]:
mystem = Mystem()
mongo_helper = NewsMongoHelper()
news = mongo_helper.get_news(config.pravda_collection)
articles = [item['text'] for item in news]

In [7]:
stop_words = [u'', u' ', u'-', '\n', u'–', u'это', u'еще', u'него', u'сказать', u'а', u'ж', u'нее', u'со', u'без', u'же', u'ней', 
      u'совсем', u'более', u'жизнь', u'нельзя', u'так', u'больше', u'за', u'нет', 
      u'такой', u'будет', u'зачем', u'ни', u'там', u'будто', u'здесь', u'нибудь', u'тебя', 
      u'бы', u'и', u'никогда', u'тем', u'был', u'из', u'ним', u'теперь', u'была', u'из-за', 
      u'них', u'то', u'были', u'или', u'ничего', u'тогда', u'было', u'им', u'но', u'того', 
      u'быть', u'иногда', u'ну', u'тоже', u'в', u'их', u'о', u'только', u'вам', u'к', u'об', 
      u'том', u'вас', u'кажется', u'один', u'тот', u'вдруг', u'как', u'он', u'три', u'ведь', 
      u'какая', u'она', u'тут', u'во', u'какой', u'они', u'ты', u'вот', u'когда', u'опять', 
      u'у', u'впрочем', u'конечно', u'от', u'уж', u'все', u'которого', u'перед', u'уже', u'всегда', 
      u'которые', u'по', u'хорошо', u'всего', u'кто', u'под', u'хоть', u'всех', u'куда', u'после',
      u'чего', u'всю', u'ли', u'потом', u'человек', u'вы', u'лучше', u'потому', u'чем', u'г', u'между', 
      u'почти', u'через', u'где', u'меня', u'при', u'что', u'говорил', u'мне', u'про', u'чтоб', u'да', 
      u'много', u'раз', u'чтобы', u'даже', u'может', u'разве', u'чуть', u'два', u'можно', u'с', u'эти', 
      u'для', u'мой', u'сам', u'этого', u'до', u'моя', u'свое', u'этой', u'другой', u'мы', u'свою', 
      u'этом', u'его', u'на', u'себе', u'этот', u'ее', u'над', u'себя', u'эту', u'ей', u'надо', u'сегодня', 
      u'я', u'ему', u'наконец', u'сейчас', 'если', u'нас', 'есть', u'не', u'также']
    
stop_words = stop_words + nltk.corpus.stopwords.words('russian')
stop_words = list(set(stop_words))

punctuation_regex = re.compile('[%s]' % re.escape(string.punctuation))

def get_stop_words_from_tokens(tokens, threshold):
    tokens = pd.Series(tokens)
    token_frequencies = tokens.value_counts()
    return token_frequencies[token_frequencies < threshold].index.values.tolist()

def remove_punctuation(text):
    """ Remove single punctuation entry from tokens list """
    return punctuation_regex.sub('', text) 

def lemmatized_formatter(text):
    return [item.lower().strip() for item in mystem.lemmatize(text) if item.strip() not in [u'', u' ']]

def clean_formatter(text):
    text = remove_punctuation(text)
    lemmas = lemmatized_formatter(text)
    tokens = [lemma for lemma in lemmas if not lemma in stop_words]
    return tokens

def clean_text(text):
    tokens = clean_formatter(text)
    return ' '.join(tokens)

In [8]:
class TextHelper:
    @staticmethod
    def get_tokens(text):
        return nltk.tokenize.word_tokenize(text)
    
    @staticmethod
    def get_ngrams(text, ngrams):
        tokens = get_tokens(text)
        ngrams = list(nltk.ngrams(tokens, ngrams))
        return ngrams
    
    @staticmethod
    def get_bigrams(text):
        return TextHelper.get_ngrams(text, 2)
    
    @staticmethod
    def get_trigrams(text):
        return TextHelper.get_ngrams(text, 3)   
    
class CorporaHelper:
    @staticmethod
    def get_tokens(articles):
        tokenized = [nltk.tokenize.word_tokenize(article) for article in articles]    
        tokens = [item for sublist in tokenized for item in sublist]
        return tokens
    
    @staticmethod
    def get_ngrams(articles, ngrams):
        text_tokens = [nltk.word_tokenize(article) for article in articles]
        bigrams_generators = [nltk.ngrams(tokens, ngrams) for tokens in text_tokens]
        bigrams_list = [list(bigrams) for bigrams in bigrams_generators]
        bigrams = [item for sublist in bigrams_list for item in sublist]
        return bigrams
    
    @staticmethod
    def get_bigrams(articles):
        return CorporaHelper.get_ngrams(articles, 2)
    
    @staticmethod
    def get_trigrams(articles):
        return CorporaHelper.get_ngrams(articles, 3)   

In [27]:
# articles_rdd = sc.parallelize(articles)
# articles_clean = articles_rdd.map(clean_text)
# articles_clean.cache()
# article_tokens = articles_clean.map(TextHelper.get_tokens)

In [29]:
# a = article_tokens.take(40)
b = pd.Series(a)
b


0     [российскотеррористический, войско, усиливать,...
1     [направление, касаться, нацбезопасность, опред...
2                                                    []
3     [парламентский, газета, голос, украина, среда,...
4                                                    []
5     [основатель, неформальный, интернетэнциклопеди...
6     [премьерминистр, арсений, яценюк, призывать, п...
7     [крымский, татарин, полуостров, запрещать, про...
8     [иловайск, россия, прибывать, 4, вагон, боепри...
9     [начало, 2015, год, число, преступление, терри...
10    [харьковский, губернатор, игорь, райнин, утвер...
11    [глава, русский, православный, церковь, патриа...
12    [пять, американский, скоростной, катер, willar...
13    [трехсторонний, переговоры, вопрос, организаци...
14    [американский, разведывательноаналитический, к...
15    [премьер, арсений, яценюк, поручать, привлекат...
16    [гаи, сообщать, подробность, дтп, мариуполь, у...
17    [боевик, свой, обстрел, снова, перебивать,